# 导入依赖

In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

# 第1步（可选）：添加一个名为`my_image_gen`的自定义工具。

In [2]:
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description`告诉代理这个工具的功能。
    description = 'AI绘画（图像生成）服务，输入文本描述，并返回基于文本信息绘制的图像URL。'
    # `parameters`告诉代理工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '所需图像内容的详细描述，用中文',
        'required': True
    }]
    
    def call(self, params: str, **kwargs) -> str:
        # `params`是由LLM代理生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'}, 
            ensure_ascii=False)

# 第2步：配置你正在使用的LLM。

In [6]:
llm_cfg = {
    # 使用阿里云上DashScope提供模型服务（需要提前注册开通）
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': 'sk-c8cda620bf4744f3b1146ff5449d9054',#'YOUR_DASHSCOPE_API_KEY',
    # 如果这里没有设置'api_key'，它将使用环境变量`DASHSCOPE_API_KEY`。

    # 使用与OpenAI API兼容的模型服务，例如vLLM或Ollama：
    # 'model': 'Qwen2-7B-Instruct',
    # 'model_server': 'http://localhost:8000/v1',  # api_base
    # 'api_key': 'EMPTY',

    # （可选）生成的LLM超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 第3步：创建一个代理。
这里我们使用`Assistant`代理作为示例，它能够使用工具和读取文件。

In [9]:
system_instruction = '''你是一个乐于助人的助手。
收到用户的请求后，你应该：
- 首先绘制一张图像并获取图像URL，
- 然后运行代码`request.get(image_url)`来下载图像，
- 最后从给定的文档中选择一个图像操作来处理图像。
请使用`plt.show()`显示图像。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter`是内置的代码执行工具。
#files = ['doc.pdf']  # 给机器人一个PDF文件来读取。
files = ['showImage.docx']  # 给机器人一个PDF文件来读取。
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 第4步：将代理作为聊天机器人运行。

In [10]:
messages = []  # 这存储聊天记录。
while True:
    # 例如，输入查询"画一只狗并将其旋转90度"。
    query = input('用户查询：')
    # 将用户查询添加到聊天记录中。
    messages.append({'role': 'user', 'content': query})
    response = []
    for response in bot.run(messages=messages):
        # 流式输出。
        print('机器人响应：')
        pprint.pprint(response, indent=2)
    # 将机器人响应添加到聊天记录中。
    messages.extend(response)

用户查询： 一只东北虎


2024-07-02 10:13:31,961 - split_query.py - 82 - INFO - Extracted info from query: {"information": ["一只东北虎
2024-07-02 10:13:33,892 - memory.py - 113 - INFO - {"keywords_zh": ["东北虎", "一只"], "keywords_en": ["Siberian tiger", "one"], "text": "一只东北虎"}
2024-07-02 10:13:33,893 - simple_doc_parser.py - 326 - INFO - Start parsing showImage.docx...
2024-07-02 10:13:33,933 - simple_doc_parser.py - 365 - INFO - Finished parsing showImage.docx. Time spent: 0.03966975212097168 seconds.
2024-07-02 10:13:33,934 - doc_parser.py - 114 - INFO - Start chunking showImage.docx (showImage.docx)...
2024-07-02 10:13:33,934 - doc_parser.py - 132 - INFO - Finished chunking showImage.docx (showImage.docx). Time spent: 5.602836608886719e-05 seconds.
2024-07-02 10:13:33,935 - base_search.py - 55 - INFO - all tokens: 222
2024-07-02 10:13:33,936 - base_search.py - 58 - INFO - use full ref


机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '{"prompt":"一只', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt":"一只东北虎"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt":"一只东北虎"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E4%B8%9C%E5%8C%97%E8%99%8E"}',
    'name': 'my_image_gen',
    'role': 'function'}]
机器人响应：
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt":"一只东北虎"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E4%B8%80%

2024-07-02 10:13:53,278 - utils.py - 156 - INFO - Downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx...
2024-07-02 10:13:53,280 - utils.py - 175 - INFO - Finished downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx. Time spent: 0.00046062469482421875 seconds.
2024-07-02 10:13:53,281 - code_interpreter.py - 181 - INFO - INFO: kernel process's PID = 3295


机器人响应：
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt":"一只东北虎"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E4%B8%9C%E5%8C%97%E8%99%8E"}',
    'name': 'my_image_gen',
    'role': 'function'},
  { 'content': '',
    'function_call': { 'arguments': '```py\n'
                                    'import requests\n'
                                    'from PIL import Image\n'
                                    'import matplotlib.pyplot as plt\n'
                                    'import io\n'
                                    '\n'
                                    '# 获取图像URL\n'
                                    'image_url = '
                                    '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E4%B8%9C%E5%8C%97%E8%99%8E"\n'
                                    '\n'
                                    '# 发送 

2024-07-02 10:13:55,424 - code_interpreter.py - 113 - INFO - stdout:

```
Exception reporting mode: Minimal

```


机器人响应：
[ { 'content': '',
    'function_call': { 'arguments': '{"prompt":"一只东北虎"}',
                       'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E4%B8%9C%E5%8C%97%E8%99%8E"}',
    'name': 'my_image_gen',
    'role': 'function'},
  { 'content': '',
    'function_call': { 'arguments': '```py\n'
                                    'import requests\n'
                                    'from PIL import Image\n'
                                    'import matplotlib.pyplot as plt\n'
                                    'import io\n'
                                    '\n'
                                    '# 获取图像URL\n'
                                    'image_url = '
                                    '"https://image.pollinations.ai/prompt/%E4%B8%80%E5%8F%AA%E4%B8%9C%E5%8C%97%E8%99%8E"\n'
                                    '\n'
                                    '# 发送 

KeyboardInterrupt: Interrupted by user

# 本地模型测试

In [ ]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

# 第1步（可选）：添加一个名为`my_image_gen`的自定义工具。
# @register_tool('my_image_gen')
# class MyImageGen(BaseTool):
#     # `description`告诉代理这个工具的功能。
#     description = 'AI绘画（图像生成）服务，输入文本描述，并返回基于文本信息绘制的图像URL。'
#     # `parameters`告诉代理工具有哪些输入参数。
#     parameters = [{
#         'name': 'prompt',
#         'type': 'string',
#         'description': '所需图像内容的详细描述，用中文',
#         'required': True
#     }]
    
#     def call(self, params: str, **kwargs) -> str:
#         # `params`是由LLM代理生成的参数。
#         prompt = json5.loads(params)['prompt']
#         prompt = urllib.parse.quote(prompt)
#         return json5.dumps(
#             {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'}, 
#             ensure_ascii=False)
        
# 第2步：配置你正在使用的LLM。
llm_cfg = {
    # 使用阿里云上DashScope提供模型服务（需要提前注册开通）
    #'model': 'qwen-max',
    #'model_server': 'dashscope',
    #'api_key': 'sk-c8cda620bf4744f3b1146ff5449d9054',#'YOUR_DASHSCOPE_API_KEY',
    # 如果这里没有设置'api_key'，它将使用环境变量`DASHSCOPE_API_KEY`。

    # 使用与OpenAI API兼容的模型服务，例如vLLM或Ollama：
     'model': 'Qwen2-7B-Instruct',
     'model_server': 'http://localhost:8000/v1',  # api_base
     'api_key': 'EMPTY',

    # （可选）生成的LLM超参数：
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 第3步：创建一个代理。这里我们使用`Assistant`代理作为示例，它能够使用工具和读取文件。
system_instruction = '''你是一个乐于助人的助手。
收到用户的请求后，你应该：
- 首先绘制一张图像并获取图像URL，
- 然后运行代码`request.get(image_url)`来下载图像，
- 最后从给定的文档中选择一个图像操作来处理图像。
请使用`plt.show()`显示图像。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter`是内置的代码执行工具。
#files = ['doc.pdf']  # 给机器人一个PDF文件来读取。
files = ['showImage.docx']  # 给机器人一个PDF文件来读取。
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 第4步：将代理作为聊天机器人运行。
messages = []  # 这存储聊天记录。
while True:
    # 例如，输入查询"画一只狗并将其旋转90度"。
    query = input('用户查询：')
    # 将用户查询添加到聊天记录中。
    messages.append({'role': 'user', 'content': query})
    response = []
    for response in bot.run(messages=messages):
        # 流式输出。
        print('机器人响应：')
        pprint.pprint(response, indent=2)
    # 将机器人响应添加到聊天记录中。
    messages.extend(response)

用户查询： 大水牛


2024-07-02 10:37:36,668 - split_query.py - 82 - INFO - Extracted info from query: {"information": ["大水牛
2024-07-02 10:37:37,216 - memory.py - 113 - INFO - {"keywords_zh": ["大水牛"], "keywords_en": ["buffalo", "water buffalo"], "text": "大水牛"}
2024-07-02 10:37:37,250 - simple_doc_parser.py - 324 - INFO - Read parsed showImage.docx from cache.
2024-07-02 10:37:37,250 - doc_parser.py - 114 - INFO - Start chunking showImage.docx (showImage.docx)...
2024-07-02 10:37:37,251 - doc_parser.py - 132 - INFO - Finished chunking showImage.docx (showImage.docx). Time spent: 7.128715515136719e-05 seconds.
2024-07-02 10:37:37,252 - base_search.py - 55 - INFO - all tokens: 222
2024-07-02 10:37:37,252 - base_search.py - 58 - INFO - use full ref


机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '', 'name': 'my_image_gen'},
    'role': 'assistant'}]
机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '{"', 'name': 'my_image_g

2024-07-02 10:37:42,439 - utils.py - 156 - INFO - Downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx...
2024-07-02 10:37:42,440 - utils.py - 175 - INFO - Finished downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx. Time spent: 0.00028586387634277344 seconds.
2024-07-02 10:37:42,441 - code_interpreter.py - 181 - INFO - INFO: kernel process's PID = 3798


机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '{"prompt": "大水牛"}', 'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B"}',
    'name': 'my_image_gen',
    'role': 'function'},
  {'content': '我已生成了一张描述为“大水牛”的图像。接下来，我将下载并显示这张图像。', 'role': 'assistant'},
  { 'content': '',
    'function_call': { 'arguments': '```py\n'
                                    '# 网络图片地址\n'
                                    'image_url = '
                                    "'https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B'\n"
                                    '# 发送 GET 请求下载图片\n'
                                    'response = requests.get(image_url)\n'
                                    '# 确保请求成功\n'
                                    'if response.status_code == 200:\n'
                                    '    # 使用 BytesIO 来创建一个可读的字节流\n'
                              

2024-07-02 10:37:44,437 - code_interpreter.py - 113 - INFO - stdout:

```
Exception reporting mode: Minimal

```


机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '{"prompt": "大水牛"}', 'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B"}',
    'name': 'my_image_gen',
    'role': 'function'},
  {'content': '我已生成了一张描述为“大水牛”的图像。接下来，我将下载并显示这张图像。', 'role': 'assistant'},
  { 'content': '',
    'function_call': { 'arguments': '```py\n'
                                    '# 网络图片地址\n'
                                    'image_url = '
                                    "'https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B'\n"
                                    '# 发送 GET 请求下载图片\n'
                                    'response = requests.get(image_url)\n'
                                    '# 确保请求成功\n'
                                    'if response.status_code == 200:\n'
                                    '    # 使用 BytesIO 来创建一个可读的字节流\n'
                              

2024-07-02 10:37:51,011 - utils.py - 156 - INFO - Downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx...
2024-07-02 10:37:51,012 - utils.py - 175 - INFO - Finished downloading showImage.docx to workspace/tools/code_interpreter/showImage.docx. Time spent: 0.0002982616424560547 seconds.


机器人响应：
[ { 'content': '',
    'function_call': {'arguments': '{"prompt": "大水牛"}', 'name': 'my_image_gen'},
    'role': 'assistant'},
  { 'content': '{image_url: '
               '"https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B"}',
    'name': 'my_image_gen',
    'role': 'function'},
  {'content': '我已生成了一张描述为“大水牛”的图像。接下来，我将下载并显示这张图像。', 'role': 'assistant'},
  { 'content': '',
    'function_call': { 'arguments': '```py\n'
                                    '# 网络图片地址\n'
                                    'image_url = '
                                    "'https://image.pollinations.ai/prompt/%E5%A4%A7%E6%B0%B4%E7%89%9B'\n"
                                    '# 发送 GET 请求下载图片\n'
                                    'response = requests.get(image_url)\n'
                                    '# 确保请求成功\n'
                                    'if response.status_code == 200:\n'
                                    '    # 使用 BytesIO 来创建一个可读的字节流\n'
                              